In [1]:
import cv2
from picamera2 import Picamera2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import pickle
import os
import numpy as np
os.environ["DISPLAY"] = ":0"

PALLET_CLS_ID = 0


In [2]:

model_path = "/home/pi/EZLift/src/raspberry_pi/custom_palletsonly_50epochs_edgetpu.tflite"

model = YOLO(model_path, task='detect')

In [3]:
def _hough_line_detection(img, canny_low=150, canny_high=450, hough_thresh=45, min_len=20, max_gap=7):
    # Get edges using Canny
    edges = edges = cv2.Canny(img, canny_low, canny_high)

    # Detect lines using Hough Transform
    lines = cv2.HoughLinesP(edges, 
                            rho=1,            # Distance resolution of accumulator in pixels
                            theta=np.pi / 180, # Angle resolution of accumulator in radians
                            threshold=hough_thresh, # Minimum number of votes (intersections in Hough grid cell)
                            minLineLength=min_len,  # Minimum length of line (pixels)
                            maxLineGap=max_gap)     # Maximum allowed gap between points on the same line
    return lines

In [4]:
def _get_most_conf_box(bbox, conf, cls_id):
    best_i = 0
    best_conf = 0
    for i in range(len(conf)):
        if cls_id[i] == PALLET_CLS_ID:
            if conf[i] > best_conf:
                best_conf = conf[i]
                best_i = i
    return bbox[best_i]

In [5]:
import math
def _get_angle_from_lines(lines, angle_thresh=2):

    lines_polar = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            dx = x2 - x1
            dy = y2 - y1
            angle_rad = math.atan2(-dy, dx)
            angle_deg = math.degrees(angle_rad)
            angle_deg = (angle_deg + 360) % 360
            length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            lines_polar.append((angle_deg, length))
    else:
        return 0

    lines_polar = sorted(lines_polar, key=lambda x: x[0])
    groups = []
    current_group = [lines_polar[0]]

    for angle, length in lines_polar[1:]:
        if abs(angle - current_group[-1][0]) < angle_thresh:
            current_group.append((angle, length))
        else:
            groups.append(current_group)
            current_group = [(angle, length)]
    groups.append(current_group)

    best_metric = 0
    best_angle = None
    for group in groups:
        avg_length = np.mean([length for _, length in group])
        if len(group) * avg_length**1.5 > best_metric:
            avg_angle = np.mean([angle for angle, _ in group])
            if not (avg_angle < 90 + angle_thresh and avg_angle > 90 - angle_thresh):
                best_metric = len(group) * avg_length**1.5
                best_angle = np.mean([angle for angle, _ in group])

    return best_angle

def _get_angle_error(bbox, img):

    img_cropped = img[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
    lines = _hough_line_detection(img_cropped)
    angle = _get_angle_from_lines(lines)
    if angle > 180:
        angle -= 180
    if angle > 90:
        error = angle - 180
    else:
        error = angle
    return error, lines

In [6]:
import time
import cv2

picam2 = Picamera2()
picam2.preview_configuration.main.size = (3280, 2464)
picam2.preview_configuration.main.format = "RGB888"
picam2.preview_configuration.align()
picam2.configure("preview")
picam2.start()
picam2.start(show_preview=False)

while True:
    start_time = time.time()
    
    # Capture frame-by-frame
    frame = picam2.capture_array()

    # Run YOLO inference on the frame with imgsz=192
    # results = model.predict(frame, device="tpu:0", imgsz=192)
    frame = cv2.resize(frame, (256, 256))
    results = model.predict(frame, device="tpu:0", imgsz=256)[0]
    bboxes = results.boxes.xyxy.cpu().numpy()
    conf = results.boxes.conf.cpu().numpy()
    cls_id = results.boxes.cls.cpu().numpy()
    if len(bboxes) > 0:
        bbox = _get_most_conf_box(bbox=bboxes, conf=conf, cls_id=cls_id)
        angle, lines = _get_angle_error(bbox, frame)
        print(angle)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        if lines is not None:
            for line in lines:
                if line is not None:
                    x1, y1, x2, y2 = line[0]
                    x1 += bbox[0]
                    x2 += min(bbox[0], 255)
                    y1 += bbox[1]
                    y2 += min(bbox[1], 255)
                    cv2.line(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green lines with thickness=2
    
        # Overlay the FPS on the annotated frame
        cv2.putText(frame, f"Angle: {angle:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow("Camera", frame)
    cv2.waitKey(1)

[1:14:14.535922523] [3775]  INFO Camera camera_manager.cpp:327 libcamera v0.4.0+53-29156679
[1:14:14.577695339] [3793]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[1:14:14.580329510] [3793]  WARN RPI vc4.cpp:393 Mismatch between Unicam and CamHelper for embedded data usage!
[1:14:14.581032224] [3793]  INFO RPI vc4.cpp:447 Registered camera /base/soc/i2c0mux/i2c@1/imx219@10 to Unicam device /dev/media0 and ISP device /dev/media2
[1:14:14.581096501] [3793]  INFO RPI pipeline_base.cpp:1121 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[1:14:14.597353781] [3775]  INFO Camera camera.cpp:1202 configuring streams: (0) 3264x2464-RGB888 (1) 3280x2464-SBGGR10_CSI2P
[1:14:14.600090636] [3793]  INFO RPI vc4.cpp:622 Sensor: /base/soc/i2c0mux/i2c@1/imx219@10 - Selected sensor format: 3280x2464-SBGGR10_1X10 - Selected unicam format: 3280x2464-pBAA


Loading /home/pi/EZLift/src/raspberry_pi/custom_palletsonly_50epochs_edgetpu.tflite on device 0 for TensorFlow Lite Edge TPU inference...

0: 256x256 (no detections), 29.1ms
Speed: 7.7ms preprocess, 29.1ms inference, 2.5ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 20.7ms
Speed: 5.5ms preprocess, 20.7ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 18.8ms
Speed: 1.8ms preprocess, 18.8ms inference, 1.4ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 17.3ms
Speed: 2.8ms preprocess, 17.3ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 17.7ms
Speed: 2.7ms preprocess, 17.7ms inference, 1.7ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 16.7ms
Speed: 2.0ms preprocess, 16.7ms inference, 1.7ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 16.6ms
Speed: 2.7ms preprocess, 16.

TypeError: cannot unpack non-iterable NoneType object